In [ ]:
import os
import cv2
import numpy as np
import pandas as pd

from skimage.feature import hog
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:

from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin


# Hàm để đọc ảnh từ thư mục
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            images.append(img)
            labels.append(label)
    return images, labels



In [ ]:
mask_images_folder = 'D:\HKIV\TGMT\dataset\mask\mask'
no_mask_images_folder = 'D:\\HKIV\\TGMT\\dataset\\no_mask\\no_mask'
mask_images , mask_labels  = load_images_from_folder(mask_images_folder, 1)
no_mask_images , no_mask_labels  = load_images_from_folder(no_mask_images_folder, 0)

In [ ]:
# Kết hợp dữ liệu
X_images = np.array(mask_images + no_mask_images)
y_labels = np.array(mask_labels + no_mask_labels)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X_images, y_labels, test_size=0.2, random_state=42)


In [ ]:
import numpy as np
import cv2
from sklearn.base import BaseEstimator, TransformerMixin
from skimage.feature import hog
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score


### HOG_SVM


In [ ]:
class HogTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, resize=(64, 64), orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2)):
        self.resize = resize
        self.orientations = orientations
        self.pixels_per_cell = pixels_per_cell
        self.cells_per_block = cells_per_block

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        hog_features_list = []
        for img in X:
            img_resized = cv2.resize(img, self.resize)
            hog_features = []
            for channel in cv2.split(img_resized):
                fd = hog(channel, orientations=self.orientations, pixels_per_cell=self.pixels_per_cell, cells_per_block=self.cells_per_block,
                         visualize=False, transform_sqrt=True, block_norm="L2")
                hog_features.append(fd)
            combined_hog_features = np.hstack(hog_features)
            hog_features_list.append(combined_hog_features)
        return np.array(hog_features_list)


In [ ]:
# Define the pipeline
pipeline_svm = Pipeline([
    ('hog', HogTransformer()),
    ('svm', SVC())
])

# Define the parameter grid
param_grid_svm = {
    'hog__orientations': [8, 9, 12],
    'hog__pixels_per_cell': [(8, 8), (16, 16)],
    'hog__cells_per_block': [(2, 2), (3, 3), (4, 4)],
    'hog__resize': [(64, 64), (128, 128)]
}

# Set up GridSearchCV
grid_search = GridSearchCV(pipeline_svm, param_grid_svm, cv=5)

# Train the model
grid_search.fit(X_train, y_train)

# Print the best parameters
print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

# Đánh giá trên tập kiểm tra
best_model = grid_search.best_estimator_
test_score = best_model.score(X_test, y_test)
print("Test score:", test_score)

Best parameters: {'hog__cells_per_block': (4, 4), 'hog__orientations': 8, 'hog__pixels_per_cell': (16, 16), 'hog__resize': (128, 128)}
Best score: 0.9590909090909092
Test score: 0.9563636363636364


In [ ]:
pd.DataFrame(grid_search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_hog__cells_per_block,param_hog__orientations,param_hog__pixels_per_cell,param_hog__resize,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,10.920136,1.115930,3.021584,0.346585,"(2, 2)",8,"(8, 8)","(64, 64)","{'hog__cells_per_block': (2, 2), 'hog__orienta...",0.936364,0.936364,0.950000,0.945455,0.961364,0.945909,0.009360,27
1,107.724377,13.906760,52.960223,9.394070,"(2, 2)",8,"(8, 8)","(128, 128)","{'hog__cells_per_block': (2, 2), 'hog__orienta...",0.945455,0.961364,0.950000,0.943182,0.961364,0.952273,0.007741,19
2,3.028444,0.180977,0.817346,0.062425,"(2, 2)",8,"(16, 16)","(64, 64)","{'hog__cells_per_block': (2, 2), 'hog__orienta...",0.911364,0.925000,0.918182,0.931818,0.915909,0.920455,0.007187,32
3,13.512294,0.900596,3.587474,0.070012,"(2, 2)",8,"(16, 16)","(128, 128)","{'hog__cells_per_block': (2, 2), 'hog__orienta...",0.945455,0.950000,0.954545,0.954545,0.961364,0.953182,0.005301,16
4,9.668638,0.245107,2.688195,0.114676,"(2, 2)",9,"(8, 8)","(64, 64)","{'hog__cells_per_block': (2, 2), 'hog__orienta...",0.940909,0.936364,0.952273,0.952273,0.965909,0.949545,0.010305,25
5,109.378301,42.736811,47.219951,12.059522,"(2, 2)",9,"(8, 8)","(128, 128)","{'hog__cells_per_block': (2, 2), 'hog__orienta...",0.943182,0.956818,0.950000,0.952273,0.961364,0.952727,0.006166,18
6,3.109658,0.111418,0.823840,0.016249,"(2, 2)",9,"(16, 16)","(64, 64)","{'hog__cells_per_block': (2, 2), 'hog__orienta...",0.918182,0.922727,0.929545,0.931818,0.918182,0.924091,0.005677,29
7,13.731200,0.234409,3.659591,0.040739,"(2, 2)",9,"(16, 16)","(128, 128)","{'hog__cells_per_block': (2, 2), 'hog__orienta...",0.940909,0.940909,0.954545,0.952273,0.968182,0.951364,0.010123,21
8,10.956202,0.161594,3.163296,0.074707,"(2, 2)",12,"(8, 8)","(64, 64)","{'hog__cells_per_block': (2, 2), 'hog__orienta...",0.931818,0.938636,0.954545,0.952273,0.961364,0.947727,0.010852,26
9,108.631678,18.902908,44.970814,9.285983,"(2, 2)",12,"(8, 8)","(128, 128)","{'hog__cells_per_block': (2, 2), 'hog__orienta...",0.945455,0.956818,0.947727,0.945455,0.959091,0.950909,0.005856,22


In [ ]:
results_svm = grid_search.cv_results_

# Đánh giá từng bộ tham số trên tập test
for i in range(len(results_svm['params'])):
    params = results_svm['params'][i]
    resize_dim = params['hog__resize']
    orientations = params['hog__orientations']
    pixels_per_cell = params['hog__pixels_per_cell']
    cells_per_block = params['hog__cells_per_block']

    # Tiền xử lý và trích xuất HOG cho tập huấn luyện và tập test
    hog_transformer = HogTransformer(resize=resize_dim, orientations=orientations, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block)
    X_train_hog = hog_transformer.transform(X_train)
    X_test_hog = hog_transformer.transform(X_test)

    # Cập nhật và huấn luyện mô hình với tham số hiện tại
    clf = SVC()
    clf.fit(X_train_hog, y_train)

    # Đánh giá trên tập test
    test_score = clf.score(X_test_hog, y_test)
    print(f"Parameters: {params}")
    print(f"Validation Score: {results_svm['mean_test_score'][i]}")
    print(f"Test Score: {test_score}\n")

Parameters: {'hog__cells_per_block': (2, 2), 'hog__orientations': 8, 'hog__pixels_per_cell': (8, 8), 'hog__resize': (64, 64)}
Validation Score: 0.9459090909090909
Test Score: 0.9527272727272728

Parameters: {'hog__cells_per_block': (2, 2), 'hog__orientations': 8, 'hog__pixels_per_cell': (8, 8), 'hog__resize': (128, 128)}
Validation Score: 0.9522727272727272
Test Score: 0.96

Parameters: {'hog__cells_per_block': (2, 2), 'hog__orientations': 8, 'hog__pixels_per_cell': (16, 16), 'hog__resize': (64, 64)}
Validation Score: 0.9204545454545455
Test Score: 0.92

Parameters: {'hog__cells_per_block': (2, 2), 'hog__orientations': 8, 'hog__pixels_per_cell': (16, 16), 'hog__resize': (128, 128)}
Validation Score: 0.9531818181818181
Test Score: 0.9527272727272728

Parameters: {'hog__cells_per_block': (2, 2), 'hog__orientations': 9, 'hog__pixels_per_cell': (8, 8), 'hog__resize': (64, 64)}
Validation Score: 0.9495454545454546
Test Score: 0.9454545454545454

Parameters: {'hog__cells_per_block': (2, 2), 

### HOG_KNN

In [ ]:
# Thiết lập pipeline
pipeline_knn = Pipeline([
    ('hog', HogTransformer()),  # Trích xuất đặc trưng HOG
    ('classifier', KNeighborsClassifier())  # Bộ phân loại SVM
])

# Định nghĩa các tham số grid search
param_grid_knn = {
     'hog__orientations': [8, 9, 12],
    'hog__pixels_per_cell': [(8, 8), (16, 16)],
    'hog__cells_per_block': [(2, 2), (3, 3), (4, 4)],
    'hog__resize': [(64, 64), (128, 128)]
}

# Grid search
grid_search = GridSearchCV(pipeline_knn, param_grid_knn, cv=5)
grid_search.fit(X_train, y_train)

print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

# Đánh giá trên tập kiểm tra
best_model = grid_search.best_estimator_
test_score = best_model.score(X_test, y_test)
print("Test score:", test_score)

Best parameters: {'hog__cells_per_block': (4, 4), 'hog__orientations': 12, 'hog__pixels_per_cell': (16, 16), 'hog__resize': (128, 128)}
Best score: 0.93
Test score: 0.9236363636363636


In [ ]:
pd.DataFrame(grid_search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_hog__cells_per_block,param_hog__orientations,param_hog__pixels_per_cell,param_hog__resize,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,4.185498,0.117095,1.305130,0.154586,"(2, 2)",8,"(8, 8)","(64, 64)","{'hog__cells_per_block': (2, 2), 'hog__orienta...",0.877273,0.893182,0.909091,0.870455,0.879545,0.885909,0.013742,16
1,18.220696,0.324920,5.046279,0.056653,"(2, 2)",8,"(8, 8)","(128, 128)","{'hog__cells_per_block': (2, 2), 'hog__orienta...",0.795455,0.811364,0.781818,0.797727,0.800000,0.797273,0.009469,35
2,2.582770,0.076534,0.656757,0.026948,"(2, 2)",8,"(16, 16)","(64, 64)","{'hog__cells_per_block': (2, 2), 'hog__orienta...",0.825000,0.809091,0.868182,0.815909,0.831818,0.830000,0.020600,30
3,9.786771,0.084822,2.549512,0.041797,"(2, 2)",8,"(16, 16)","(128, 128)","{'hog__cells_per_block': (2, 2), 'hog__orienta...",0.895455,0.868182,0.897727,0.881818,0.879545,0.884545,0.010890,17
4,4.688266,0.066373,1.352347,0.026547,"(2, 2)",9,"(8, 8)","(64, 64)","{'hog__cells_per_block': (2, 2), 'hog__orienta...",0.879545,0.890909,0.900000,0.870455,0.890909,0.886364,0.010265,15
5,18.719540,0.103621,5.198573,0.070547,"(2, 2)",9,"(8, 8)","(128, 128)","{'hog__cells_per_block': (2, 2), 'hog__orienta...",0.786364,0.790909,0.795455,0.820455,0.800000,0.798636,0.011818,34
6,2.698010,0.070213,0.666469,0.026170,"(2, 2)",9,"(16, 16)","(64, 64)","{'hog__cells_per_block': (2, 2), 'hog__orienta...",0.850000,0.838636,0.859091,0.831818,0.852273,0.846364,0.009812,24
7,10.083801,0.148170,2.633316,0.041709,"(2, 2)",9,"(16, 16)","(128, 128)","{'hog__cells_per_block': (2, 2), 'hog__orienta...",0.902273,0.877273,0.890909,0.890909,0.893182,0.890909,0.008003,13
8,5.338366,0.501452,1.449051,0.100893,"(2, 2)",12,"(8, 8)","(64, 64)","{'hog__cells_per_block': (2, 2), 'hog__orienta...",0.868182,0.904545,0.897727,0.868182,0.902273,0.888182,0.016477,14
9,19.685486,0.108444,5.619478,0.073382,"(2, 2)",12,"(8, 8)","(128, 128)","{'hog__cells_per_block': (2, 2), 'hog__orienta...",0.781818,0.793182,0.775000,0.784091,0.797727,0.786364,0.008131,36


In [ ]:
results_knn = grid_search.cv_results_

# Đánh giá từng bộ tham số trên tập test
for i in range(len(results_knn['params'])):
    params = results_knn['params'][i]
    resize_dim = params['hog__resize']
    orientations = params['hog__orientations']
    pixels_per_cell = params['hog__pixels_per_cell']
    cells_per_block = params['hog__cells_per_block']

    # Tiền xử lý và trích xuất HOG cho tập huấn luyện và tập test
    hog_transformer = HogTransformer(resize=resize_dim, orientations=orientations, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block)
    X_train_hog = hog_transformer.transform(X_train)
    X_test_hog = hog_transformer.transform(X_test)

    # Cập nhật và huấn luyện mô hình với tham số hiện tại
    clf = KNeighborsClassifier()
    clf.fit(X_train_hog, y_train)

    # Đánh giá trên tập test
    test_score = clf.score(X_test_hog, y_test)
    print(f"Parameters: {params}")
    print(f"Validation Score: {results_knn['mean_test_score'][i]}")
    print(f"Test Score: {test_score}\n")

Parameters: {'hog__cells_per_block': (2, 2), 'hog__orientations': 8, 'hog__pixels_per_cell': (8, 8), 'hog__resize': (64, 64)}
Validation Score: 0.885909090909091
Test Score: 0.8581818181818182

Parameters: {'hog__cells_per_block': (2, 2), 'hog__orientations': 8, 'hog__pixels_per_cell': (8, 8), 'hog__resize': (128, 128)}
Validation Score: 0.7972727272727272
Test Score: 0.8290909090909091

Parameters: {'hog__cells_per_block': (2, 2), 'hog__orientations': 8, 'hog__pixels_per_cell': (16, 16), 'hog__resize': (64, 64)}
Validation Score: 0.8299999999999998
Test Score: 0.8327272727272728

Parameters: {'hog__cells_per_block': (2, 2), 'hog__orientations': 8, 'hog__pixels_per_cell': (16, 16), 'hog__resize': (128, 128)}
Validation Score: 0.8845454545454545
Test Score: 0.9054545454545454

Parameters: {'hog__cells_per_block': (2, 2), 'hog__orientations': 9, 'hog__pixels_per_cell': (8, 8), 'hog__resize': (64, 64)}
Validation Score: 0.8863636363636364
Test Score: 0.889090909090909

Parameters: {'hog__